# ISO TC/211 Harmonised Model Controls

Connect to the EA app and model repository

In [2]:
from Parameters import *
from EAConnect import *
from HM_Controls import *
import sys
import pandas as pd

# Open EA Repository and find Model
eaApp = openEAapp()
eaRepo = openEArepo(eaApp,repo_path)
try:
    omMod = eaRepo.Models.GetByName(modelName)
    printTS('Model "' + modelName + '" found with PackageGUID ' + omMod.PackageGUID )
except Exception as e:
    printTS('Model  "' + modelName + '" not found!')
    closeEA(eaRepo)
    sys.exit()
printTS('Number of main packages: ' + str(omMod.Packages.Count))
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

2024-04-15 12:00:23   Hi EA - are you there? 
2024-04-15 12:00:23   I am here
2024-04-15 12:00:23   Hi EA - Please open this repository: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea
2024-04-15 12:00:25   OK! Repository C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea is ready!
2024-04-15 12:00:25   Model "Conceptual Models" found with PackageGUID {7B6B28E9-C583-4363-9E9C-F37A37AE06C9}
2024-04-15 12:00:26   Number of main packages: 5


## Select a package in EA before continuing

In [3]:
thePackage = eaRepo.GetTreeSelectedPackage()
printTS('Selected package name: ' + thePackage.Name)

2024-04-15 12:00:38   Selected package name: ISO/AWI TS 19124-2 Edition 1


**Duplicates:**

Check that there are no duplicate element names

In [4]:
df = duplicateElements(thePackage)
non_unique = df[df.duplicated(subset=['ElementName'], keep=False)]
errCount = len(non_unique)
printTS('Number of errors: ' + str(errCount))

if errCount > 0:
    print('')
    print('Duplicate elements: ')
    for index, row in non_unique.iterrows():
        combined_string = f"{row['ElementName']}"
        print(combined_string)

2024-04-15 12:00:49   Number of errors: 0


**Data type references:**

Check that all data types are references to an element.
Missing references to primitive data types are fixed in the script.

The results are stored in a data frame for further use.

In [5]:
#df = pd.DataFrame(columns=['FullPath','Package','Element','Property','DependentPackage','DependentElement','GUID'])
df = listClassifiers(eaRepo,thePackage)
noRef = df[df['GUID'].isna()]
errCount = len(noRef)
printTS('')
printTS('Number of errors: ' + str(errCount))

print('')
print('Attributes without reference:')
for index, row in noRef.iterrows():
    combined_string = f"{row['Package']}.{row['Element']}.{row['Property']} (Data type:{row['DependentElement']})"
    print(combined_string)
 

2024-04-15 12:00:58   ERROR|Missing data type connection for attribute :CA_PlatformInfo.passDirection (Data type: CA_OrbitPassDirection)
2024-04-15 12:00:58   ERROR|Missing data type connection for attribute :CA_SARDataAcquiPara.imagingMode (Data type: CA_SARImagingMode)
2024-04-15 12:00:58   
2024-04-15 12:00:58   Number of errors: 2

Attributes without reference:
ISO/AWI TS 19124-2 Edition 1.CA_PlatformInfo.passDirection (Data type:CA_OrbitPassDirection)
ISO/AWI TS 19124-2 Edition 1.CA_SARDataAcquiPara.imagingMode (Data type:CA_SARImagingMode)


**Dependencies:**

Check which packages data types and relations are connected to.

In [6]:
dfCounts = df.groupby(['DependentPackage']).size()
print(dfCounts)

DependentPackage
ISO TC211.ISO 19103 Conceptual schema language.ISO 19103 Edition 1.Measure types                                            3
ISO TC211.ISO 19103 Conceptual schema language.ISO 19103 Edition 1.Primitive types.Date and Time                            1
ISO TC211.ISO 19103 Conceptual schema language.ISO 19103 Edition 1.Primitive types.Numerics                                24
ISO TC211.ISO 19103 Conceptual schema language.ISO 19103 Edition 1.Primitive types.Text                                    42
ISO TC211.ISO 19103 Conceptual schema language.ISO 19103 Edition 1.Primitive types.Truth                                    1
ISO TC211.ISO 19124 Calibration and validation of remote sensing data and derived products.ISO TS 19124-1 Edition 1         3
ISO TC211.ISO 19124 Calibration and validation of remote sensing data and derived products.ISO/AWI TS 19124-2 Edition 1    39
dtype: int64


**Dependencies:**

List elements from a specific package 

In [11]:
strFilter = 'ISO TC211.ISO 19124 Calibration and validation of remote sensing data and derived products.ISO TS 19124-1'

df_cleaned = df.dropna(subset=['DependentPackage'])
filtered_df = df_cleaned[df_cleaned['DependentPackage'].str.startswith(strFilter)]

if len(filtered_df) > 0:
    print('Refered elements from package "' + strFilter + '*"')
    for index, row in filtered_df.iterrows():
        print(f"{row['Element']}")
else:
    print('No elements found!')    

Refered elements from package "ISO TC211.ISO 19124 Calibration and validation of remote sensing data and derived products.ISO TS 19124-1*"
CA_SARDataCalVal
CA_SARDataQualEvalIndex
CA_SARMultiDimensionalProduct


**Definitions:**

Check that all classes, attributes and navigable associations have definitions

In [12]:
#defDf = pd.DataFrame(columns=['Type','PackageName','ElementName','PropertyName','Supplier'])
defDf = listMissingDefinitions(eaRepo,thePackage)

errCount = len(defDf)
printTS('')
printTS('Number of errors: ' + str(errCount))

noDef = defDf[defDf['PropertyName'].isna() & defDf['Supplier'].isna()]
if len(noDef) > 0:
    print('')
    print('Elements without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Attribute']
if len(noDef) > 0:
    print('')
    print('Attributes without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Role name']
if len(noDef) > 0:
    print('')
    print('Navigable association ends without role name (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']} towards {row['Supplier']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Role']
if len(noDef) > 0:
    print('')
    print('Navigable association ends without definition (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        if row['PropertyName'] != None:
            combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']} towards {row['Supplier']}"
        else:
            combined_string = f"{row['PackageName']}.{row['ElementName']} towards {row['Supplier']}"
        print(combined_string)


2024-04-15 12:03:30   ERROR|Missing role name for Navigable association in package ISO/AWI TS 19124-2 Edition 1: CA_SARDataCal towards CA_SARDataGeoCal
2024-04-15 12:03:30   ERROR|Missing role definition for Navigable association in package ISO/AWI TS 19124-2 Edition 1: CA_SARDataCal towards CA_SARDataGeoCal
2024-04-15 12:03:30   ERROR|Missing role name for Navigable association in package ISO/AWI TS 19124-2 Edition 1: CA_SARDataCal towards CA_SARDataRadioCal
2024-04-15 12:03:30   ERROR|Missing role definition for Navigable association in package ISO/AWI TS 19124-2 Edition 1: CA_SARDataCal towards CA_SARDataRadioCal
2024-04-15 12:03:30   ERROR|Missing role name for Navigable association in package ISO/AWI TS 19124-2 Edition 1: CA_SARDataCal towards CA_SARDataChaParaCal
2024-04-15 12:03:30   ERROR|Missing role definition for Navigable association in package ISO/AWI TS 19124-2 Edition 1: CA_SARDataCal towards CA_SARDataChaParaCal
2024-04-15 12:03:30   ERROR|Missing role name for Navigabl

**Diagram representation:**

Check that all elements are shown in at least one diagram

In [13]:
dfE = elementsInDiagrams(thePackage)
errCount = len(dfE)
printTS('')
printTS('Number of errors: ' + str(errCount))

if errCount > 0:
    print('')
    print('Elements that are not in any diagram:')
    for index, row in dfE.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}"
        print(combined_string)    


2024-04-15 12:03:37   Diagram: 19124_2_01 (12 objects)
2024-04-15 12:03:37   Diagram: 19124_2_02 (9 objects)
2024-04-15 12:03:37   Diagram: 19124_2_03 (6 objects)
2024-04-15 12:03:37   Diagram: 19124_2_07 (2 objects)
2024-04-15 12:03:37   Diagram: 19124_2_04 (14 objects)
2024-04-15 12:03:37   Diagram: 19124_2_05 (5 objects)
2024-04-15 12:03:37   Diagram: 19124_2_06 (3 objects)
2024-04-15 12:03:37   
2024-04-15 12:03:37   Number of errors: 0


**Diagram layout:**

Setting diagram fonts to Cambria and hiding "isSubstitutable" labels

In [14]:
recDiagramCleaning(thePackage)

2024-04-15 12:03:42   Diagram count: 7
2024-04-15 12:03:42   Diagram: 19124_2_01 (12 objects)
2024-04-15 12:03:42   Diagram: 19124_2_02 (9 objects)
2024-04-15 12:03:42   Diagram: 19124_2_03 (6 objects)
2024-04-15 12:03:42   Diagram: 19124_2_07 (2 objects)
2024-04-15 12:03:42   Diagram: 19124_2_04 (14 objects)
2024-04-15 12:03:42   Diagram: 19124_2_05 (5 objects)
2024-04-15 12:03:42   Diagram: 19124_2_06 (3 objects)
